## Reverse Game of Life

We need to try to predict how the result was. Also, to apply the genetic algorithm, we must mutate the final result until we reach what we want(start pattern).

In [5]:
# Imports
import pandas as pd
import numpy as np

In [9]:
# Import datasets
train_pd = pd.read_csv("resources/train.csv")
test_pd = pd.read_csv("resources/test.csv")
sample_submission_pd = pd.read_csv("resources/sampleSubmission.csv")

# Adjust
# Pandas dataframe to numpy
train = train_pd.values
test = test_pd.values

# tests = np.random.binomial(5, 0.5, size=2)
# print(tests)

In [11]:
# test[x][0] = game id
# test[x][1] = id delta - the number of steps between the start and stop boards
print(train[0])

[1 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [12]:
# Aux Functions

"""
Function to count how many neighbors are alive. For that, we need an array neigh_pos with all 
possible positions of neighbors

(-1,-1) (0,-1) (1,-1) 

(-1,0)   cell  (1,0)

(-1,1)  (0,1)  (1,1)


Parameters: field = map, x and y = positions of the cell
"""

neigh_pos = [
    (-1,-1), (0,-1), (1,-1),
    (-1,0), (1,0),
    (-1,1), (0,1), (1,1)
]

def count_neighbors(field, x, y):
    count = 0
    size = len(field)
    
    for pos in neigh_pos:
        x2 = pos[0] + x
        y2 = pos[1] + y
        
        if x2 < size and y2 < size:
            if field[x2][y2] == 1:
                ++count
    return count

"""
This next function will transform the field cell by cell according to the 4 rules of the game:
    1. Each live cell with one or no neighbors dies, as if by solitude.
    2. Each live cell with four or more neighbors dies, as if by overpopulation.
    3. Each live cell with two or three neighbors survives.
    4. Each dead cell with three neighbors becomes populated.
"""
def apply_rules(field, no_mutations=1):
    size = len(field)
    tmp = field
    
    for _ in range(no_mutations):
        new = np.zeros((size, size), dtype="uint8")
        
        for i in range(size):
            for i in range(size):
                no_neighbors = count_neighbors(tmp, i, j)
                
                # Mutations based on rules - on the new field all cells are already dead=0.
                # if it was a live cell and has 2 neighbors, lives
                if tmp[i][j] == 1 and no_neighbors == 2:
                    new[i][j] = 1
                # If it has 3 neighbors, it doesn't matter if it's dead or not, becomes live cell.
                if no_neighbors == 3:
                    new[i][j] = 1
                # Less than 2 or more than 3 becomes dead, no need to change values.
        
        # tmp receives the mutations
        tmp = new
    return tmp


"""
This function will create a population randomly in a 20x20 array
"""
def generate_population(size, random_state=-1):
    if random_state != -1:
        np.random.seed(random_state)
    initial_states = np.split(np.random.binomial(1, 0.5, (20 * size, 20)).astype('uint8'), size)
    return [make_move(state, 5) for state in initial_states]